In [137]:
import pandas as pd #for manipulating the csv data
import numpy as np #for mathematical calculation

In [138]:
train_data_m = pd.read_csv("golf_data.csv")

In [139]:
train_data_m.head()

,Ambiente,Temperatura,Humedad,Viento,Jugar
0,S,A,A,N,N
1,S,A,A,S,N
2,N,A,A,N,S
3,Ll,M,A,N,S
4,Ll,B,N,N,S


In [140]:
def calc_total_entropy(train_data, label, class_list):
    total_row = train_data.shape[0]
    total_entr = 0
    
    for c in class_list:
        total_class_count = train_data[train_data[label] == c].shape[0]
        total_class_entr = - (total_class_count/total_row)*np.log2(total_class_count/total_row) 
        total_entr += total_class_entr
    
    return total_entr

In [141]:
def calc_entropy(feature_value_data, label, class_list):
    class_count = feature_value_data.shape[0]
    entropy = 0
    
    for c in class_list:
        label_class_count = feature_value_data[feature_value_data[label] == c].shape[0]
    
        entropy_class = 0
        if label_class_count != 0:
            probability_class = label_class_count/class_count
            entropy_class = - probability_class * np.log2(probability_class) 
        
        entropy += entropy_class
        
    return entropy

In [142]:
def calc_info_gain(feature_name, train_data, label, class_list):
    feature_value_list = train_data[feature_name].unique()
    total_row = train_data.shape[0]
    feature_info = 0.0
    
    for feature_value in feature_value_list:
        feature_value_data = train_data[train_data[feature_name] == feature_value]
        feature_value_count = feature_value_data.shape[0]
        feature_value_entropy = calc_entropy(feature_value_data, label, class_list)
        feature_value_probability = feature_value_count/total_row
        feature_info += feature_value_probability * feature_value_entropy
        
    return calc_total_entropy(train_data, label, class_list) - feature_info

In [143]:
def find_most_informative_feature(train_data, label, class_list):
    feature_list = train_data.columns.drop(label)
    max_info_gain = -1
    max_info_feature = None
    
    for feature in feature_list:  
        feature_info_gain = calc_info_gain(feature, train_data, label, class_list)
        if max_info_gain < feature_info_gain:
            max_info_gain = feature_info_gain
            max_info_feature = feature
            
    return max_info_feature

In [144]:
def generate_sub_tree(feature_name, train_data, label, class_list):
    feature_value_count_dict = train_data[feature_name].value_counts(sort=False)
    tree = {}
    
    for feature_value, count in feature_value_count_dict.iteritems():
        feature_value_data = train_data[train_data[feature_name] == feature_value]
        
        assigned_to_node = False
        for c in class_list:
            class_count = feature_value_data[feature_value_data[label] == c].shape[0]

            if class_count == count:
                tree[feature_value] = c
                print(feature_value_data, c)
                train_data = train_data[train_data[feature_name] != feature_value]
                assigned_to_node = True
        if not assigned_to_node:
            tree[feature_value] = "?"
            
    return tree, train_data

In [145]:
import pandas as pd

# Carga los datos desde el archivo CSV
data = pd.read_csv('golf_data.csv')

feature_name = 'Ambiente'
label = 'Jugar'
class_list = data[label].unique()

def generate_sub_tree(feature_name, train_data, label, class_list):
    feature_value_count_dict = train_data[feature_name].value_counts(sort=False)
    tree = {}
    
    for feature_value, count in feature_value_count_dict.iteritems():
        feature_value_data = train_data[train_data[feature_name] == feature_value]
        
        assigned_to_node = False
        for c in class_list:
            class_count = feature_value_data[feature_value_data[label] == c].shape[0]

            if class_count == count:
                tree[feature_value] = c
                train_data = train_data[train_data[feature_name] != feature_value]
                assigned_to_node = True
        if not assigned_to_node:
            tree[feature_value] = "?"
            
    return tree, train_data

arbol_generado, datos_restantes = generate_sub_tree(feature_name, data, label, class_list)

def imprimir_arbol_generado(arbol, nivel=0):
    for clave, valor in arbol.items():
        indentacion = "  " * nivel
        if valor == "?":
            print(f"{indentacion}{clave}: {valor}")
        else:
            print(f"{indentacion}{clave} --> {valor}")
            if isinstance(valor, dict):
                imprimir_arbol_generado(valor, nivel + 1)

imprimir_arbol_generado(arbol_generado)



S: ?
N --> S
Ll: ?


In [146]:
def make_tree(root, prev_feature_value, train_data, label, class_list):
    if train_data.shape[0] != 0:
        max_info_feature = find_most_informative_feature(train_data, label, class_list)
        tree, train_data = generate_sub_tree(max_info_feature, train_data, label, class_list)
        next_root = None
        
        if prev_feature_value != None:
            print("prev", prev_feature_value)
            root[prev_feature_value] = dict()
            root[prev_feature_value][max_info_feature] = tree
            print("max", max_info_feature)
            next_root = root[prev_feature_value][max_info_feature]
        else:
            root[max_info_feature] = tree
            next_root = root[max_info_feature]
        
        for node, branch in list(next_root.items()):
            if branch == "?":
                feature_value_data = train_data[train_data[max_info_feature] == node]
                print("max", max_info_feature,"node", node)
                print(feature_value_data)
                make_tree(next_root, node, feature_value_data, label, class_list)

In [147]:
def id3(train_data_m, label):
    train_data = train_data_m.copy()
    tree = {}
    class_list = train_data[label].unique()
    make_tree(tree, None, train_data, label, class_list)
    
    return tree

In [148]:
def predict(tree, instance):
    if not isinstance(tree, dict):
        return tree
    else:
        root_node = next(iter(tree))
        feature_value = instance[root_node]
        if feature_value in tree[root_node]:
            return predict(tree[root_node][feature_value], instance)
        else:
            return None

In [149]:
def evaluate(tree, test_data_m, label):
    correct_preditct = 0
    wrong_preditct = 0
    for index, row in test_data_m.iterrows():
        result = predict(tree, test_data_m.iloc[index])
        if result == test_data_m[label].iloc[index]:
            correct_preditct += 1
        else:
            wrong_preditct += 1
    accuracy = correct_preditct / (correct_preditct + wrong_preditct)
    return accuracy

In [150]:
tree = id3(train_data_m, 'Jugar')
tree

max Ambiente node S
   Ambiente Temperatura Humedad Viento Jugar
0         S           A       A      N     N
1         S           A       A      S     N
7         S           M       A      N     N
8         S           B       N      N     S
10        S           M       N      S     S
prev S
max Humedad
max Ambiente node Ll
   Ambiente Temperatura Humedad Viento Jugar
3        Ll           M       A      N     S
4        Ll           B       N      N     S
5        Ll           B       N      S     N
9        Ll           M       N      N     S
13       Ll           M       A      S     N
prev Ll
max Viento


{'Ambiente': {'S': {'Humedad': {'A': 'N', 'N': 'S'}},
  'N': 'S',
  'Ll': {'Viento': {'N': 'S', 'S': 'N'}}}}

In [151]:
accuracy = evaluate(tree, test_data_m, 'Play Tennis')
print("accuracy:", accuracy)

NameError: name 'test_data_m' is not defined